In [52]:
!pip install "numpy<2"
!pip install scikit-surprise
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader, SVD, accuracy

In [53]:
from google.colab import files

# Upload the ratings.csv file
uploaded = files.upload()  # Choose 'ratings.csv'

Saving movie.zip to movie (5).zip


In [54]:
!unzip movie.zip

Archive:  movie.zip
replace ml-100k/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: A]ll
  inflating: ml-100k/README          
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [55]:


# 2. Load Dataset
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|',
    names=['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
           'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
           'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')

# 3. Build user-item matrix for item-based CF
user_item_matrix = ratings.pivot(index='user_id', columns='item_id', values='rating')

# 4. Build Item Similarity Matrix
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# 5. Item-Based Recommendation Function
def recommend_item_based(user_id, user_item_matrix, item_similarity, top_n=5):
    user_ratings = user_item_matrix.loc[user_id]
    weighted_scores = item_similarity.dot(user_ratings.fillna(0))
    sim_sum = np.abs(item_similarity).sum(axis=1)
    sim_sum[sim_sum == 0] = 1e-8
    scores = weighted_scores / sim_sum
    already_rated = user_ratings.dropna().index
    recommendations = {
        movieId: score for movieId, score in zip(user_item_matrix.columns, scores)
        if movieId not in already_rated
    }
    top_movie_ids = sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]
    top_movie_ids_int = [int(mid) for mid in top_movie_ids]
    recommended_titles = movies[movies['item_id'].isin(top_movie_ids_int)][['title']]
    return recommended_titles

# 6. Train SVD Model with Surprise
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)


RMSE: 0.9373


In [56]:

# 7. Get Top-N SVD Recommendations
from collections import defaultdict

def get_top_n_recommendations(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n_recommendations(predictions, n=5)

# 8. Display SVD Recommendations for a Sample User
user_id = 1
svd_recs = top_n[user_id]
svd_movie_ids = [int(iid) for iid, _ in svd_recs]
svd_titles = movies[movies['item_id'].isin(svd_movie_ids)][['title']]
print(f"\nTop 5 SVD Recommendations for User {user_id}:")
print(svd_titles.to_string(index=False))



Top 5 SVD Recommendations for User 1:
                           title
Silence of the Lambs, The (1991)
             Citizen Kane (1941)
          Cinema Paradiso (1988)
                   Brazil (1985)
               GoodFellas (1990)


In [57]:

# 9. Display Item-Based Recommendations for Same User
item_recs = recommend_item_based(user_id, user_item_matrix, item_similarity_df, top_n=5)
print(f"\nTop 5 Item-Based Recommendations for User {user_id}:")
print(item_recs.to_string(index=False))

# 10. Optional: Precision@K
def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / k



Top 5 Item-Based Recommendations for User 1:
                                    title
                    Jupiter's Wife (1994)
                           Witness (1985)
                  King of New York (1990)
                   All Things Fair (1996)
Scream of Stone (Schrei aus Stein) (1991)
